In [ ]:
def get_api_id(track_info: str, token: str, artist: str = None) -> str:
    '''Performs a query on Spotify API to get a track ID.
    See https://curl.trillworks.com/'''

    # Set up logging
    logging.basicConfig(filename='spotify_api.log', level=logging.ERROR)

    # Define headers for the HTTP request
    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/json',
        'Authorization': f'Bearer ' + token,
    }

    # Extract track name and artist from the input track_info
    track_name = track_info.split("___")[0]

    # Prepare parameters for the API request
    params = [
        ('q', track_name),
        ('type', 'track'),
    ]

    # If artist information is provided, add it to the parameters
    artist = track_info.split("___")[-1]
    if artist:
        params.append(('artist', artist))

    try:
        # Make a GET request to Spotify API with the specified parameters
        response = requests.get('https://api.spotify.com/v1/search',
                                headers=headers, params=params, timeout=5)
        response.raise_for_status()  # Raise HTTPError for bad responses

        json_data = response.json()

        # Check if the response contains track information
        if 'tracks' in json_data and 'items' in json_data['tracks']:
            results = json_data['tracks']['items']

            # Check if the specified artist is in the response
            if artist:
                for result in results:
                    if artist.strip() == result['artists'][0]['name'].strip():
                        track_id = result['id']
                        return track_id

            # If the specified artist is not found, use the first result
            if results:
                track_id = results[0]['id']
                return track_id
            else:
                logging.warning(f"No results found for track: {track_name}")

        else:
            logging.warning(f"Unexpected response format from Spotify API: {json_data}")

    except requests.exceptions.Timeout:
        logging.error("Timeout error: The Spotify API request timed out.")
    except requests.exceptions.RequestException as e:
        logging.error(f"Request error: {e}")
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")

    # Return None in case of errors
    return None

In [ ]:
import concurrent.futures
import requests

def request_get(url, headers, params):
    return requests.get(url, headers=headers, params=params)

def get_api_id_new(track_info, token, artist=None):
    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {token}',
    }
    track_name = track_info.split("___")[0]
    params = [
        ('q', track_name),
        ('type', 'track'),
    ]
    artist = track_info.split("___")[-1]
    if artist:
        params.append(('artist', artist))

    with concurrent.futures.ThreadPoolExecutor() as executor:
        res = [executor.submit(request_get, 'https://api.spotify.com/v1/search', headers=headers, params=params) for _ in range(15)]
        concurrent.futures.wait(res)

    try:
        json = res[0].result().json()
        results = json['tracks']['items']
        first_result = json['tracks']['items'][0]
        if artist:
            for result in results:
                if artist.strip() == result['artists'][0]['name'].strip():
                    track_id = result['id']
                    return track_id
        track_id = first_result['id']
        return track_id
    except:
        return None

In [ ]:
def get_song_id(row, token):
    return get_api_id_new(row['trackName'], token, artist=row['artistName'])

In [ ]:
song_id = get_api_id('Pleaser', token, artist = 'Wallows')
print(song_id)

In [ ]:
playlist = pd.read_csv('playlist_data.csv')
mask = df['trackName'].isin(playlist['trackName'])
df = df[~mask]
df.head()

In [ ]:
saved_id = pd.read_csv("D:/ADITYA/Documents/adi new/coding/spotify/saved_id.csv")
mask = df['trackName'].isin(saved_id['trackName'])
df = df[~mask]
df.head()

In [ ]:
df1 = df[['artistName','trackName']].groupby('trackName').first().reset_index()

In [ ]:
start_time = time.time()
df2['id'] = df2.apply(get_song_id, token=token, axis=1)
execution_time = time.time() - start_time

In [ ]:
if df1.empty == False:
    df1['id'] = df1.apply(get_song_id, token=token, axis=1)

In [ ]:
file_path = 'saved_id.csv'
df1.to_csv(file_path,mode='a', header=False, index=False)